In [1]:
# take production nwb spikes as input to create tensor and pandas dataframe

In [1]:
import numpy as np
# in python 3, cPickle has changed from cPickle to _pickle
import pandas as pd
import scipy.sparse as sp
import h5py

import matplotlib.pyplot as plt
%matplotlib inline

import os, sys,glob, copy
sys.path.append("/home/xiaoxuanj/work/work_allen/Ephys/code_library/ephys_code")
#sys.path.append("/Users/xiaoxuanj/work/work_allen/Ephys/code_library/ephys_code")

import nwb_adapter_spikes_newNWB_py3 as nwb_adapter_spikes

def nanunique(x):
    """np.unique get rid of nan numbers."""
    temp = np.unique(x.astype(float))
    return temp[~np.isnan(temp)]

def save_npz(matrix, filename):
    matrix_2d = matrix.reshape(matrix.shape[0], int(len(matrix.flatten())/matrix.shape[0]))
    sparse_matrix = sp.csc_matrix(matrix_2d)
    np.savez(filename, [sparse_matrix, matrix.shape])
    return 'npz file saved'

def load_npz(filename):
    """
    load npz files with sparse matrix and dimension
    output dense matrix with the correct dim
    """
    npzfile = np.load(filename, allow_pickle=True) 
    sparse_matrix = npzfile['arr_0'][0]
    ndim=npzfile['arr_0'][1]

    new_matrix_2d = np.array(sparse_matrix.todense())
    new_matrix = new_matrix_2d.reshape(ndim)
    return new_matrix

In [2]:
# production mouse
mouse_IDs = [#'306046','388523', '389262',  '408153','410344', '416856','434836', '434845',
             '415149', '412809', '412804',  '419114',  '419117', '419118', '419119', '424445',
             '415148', '416356','416861','419112','419116',
             '434488','437661','448503',
            '433891', '432105', '432104',
            '437660', '429857', '434494',
            '425597', '415148', '418196', ]  
mouseIDs = ['mouse'+m for m in mouse_IDs]

In [3]:
keys=['spontaneous', 'gabors', 'flashes', 'drifting_gratings_contrast', 'natural_movie_one_more_repeats', 'natural_movie_one_shuffled', 
      'drifting_gratings_75_repeats', 'dot_motion']

In [2]:
basepath = "/local1/work_allen/Ephys/nwb_files/production/ecephys_project_cache/"

sessions_id=pd.read_csv(basepath+'sessions_id.csv', index_col=0)
session_id_all=sessions_id[sessions_id.session_type=='brain_observatory_1.1']
mouseIDs = ['mouse'+str(i) for i in session_id_all.mouseID.values]
mouseIDs

['mouse386129',
 'mouse387858',
 'mouse388521',
 'mouse404553',
 'mouse404568',
 'mouse404551',
 'mouse404571',
 'mouse404555',
 'mouse404569',
 'mouse403407',
 'mouse412791',
 'mouse412792',
 'mouse405755',
 'mouse404570',
 'mouse404554',
 'mouse405751',
 'mouse406807',
 'mouse412794',
 'mouse412793',
 'mouse406805',
 'mouse412799',
 'mouse407972',
 'mouse406808',
 'mouse412802',
 'mouse410343',
 'mouse408152',
 'mouse412803',
 'mouse410315',
 'mouse417678',
 'mouse421529',
 'mouse421338',
 'mouse424448']

In [6]:
keys = ['spontaneous', 'gabors', 'flashes', 'drifting_gratings', 'natural_movie_three', 'natural_movie_one', 'static_gratings', 'natural_scenes']

In [22]:
#brain observatory
condi='cortex_nwb2'
#condi='all_units'
for mouse_idx, mouseID in enumerate(mouseIDs):
    print(mouse_idx, mouseID)
    #output = '/local1/work_allen/Ephys/resorted/'+mouseID
    output = '/local1/work_allen/Ephys/resorted/'+mouseID
    output_path = output+'/matrix/'

    output_meta = mouseID+'_'+condi+'_meta.csv'
    output_path = output+'/matrix/'

    #for gratings
    key = 'drifting_gratings'
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    stim_table = pd.read_csv(output_path+'stim_table_'+key+'.csv')
    oris = np.sort(stim_table.orientation.unique())[:-1]
    tfs = np.sort(stim_table.temporal_frequency.unique())[:-1]

    rep = int(np.shape(matrix_unit_binarized)[1]/(len(oris)*len(tfs)))
    spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(oris), len(tfs), int(rep), np.shape(matrix_unit_binarized)[2])).astype('uint8')

    for idx_o, o in enumerate(oris):
        for idx_t, t in enumerate(tfs):
            tmp = matrix_unit_binarized[:,np.where((stim_table['orientation']==o) & (stim_table['temporal_frequency']==t))[0],:]
            # pad 0 if there is missing rep
            if np.shape(tmp)[1]<np.shape(spikes)[-2]:
                result = np.zeros((np.shape(spikes)[0], np.shape(spikes)[-2], np.shape(spikes)[-1]))
                result[:,:tmp.shape[1],:] = tmp.astype('uint8')
                spikes[:,idx_o,idx_t,:,:] = result.astype('uint8')
            else:
                spikes[:,idx_o,idx_t,:,:] = tmp.astype('uint8')
    spikes = np.squeeze(spikes)
    save_npz(np.squeeze(spikes), output_path+'drifting_grating_'+condi+'_rep.npz')


0 mouse386129
1 mouse387858
2 mouse388521
3 mouse404553
4 mouse404568
5 mouse404551
6 mouse404571
7 mouse404555
8 mouse404569
9 mouse403407
10 mouse412791
11 mouse412792
12 mouse405755
13 mouse404570
14 mouse404554
15 mouse405751
16 mouse406807
17 mouse412794
18 mouse412793
19 mouse406805
20 mouse412799
21 mouse407972
22 mouse406808
23 mouse412802
24 mouse410343
25 mouse408152
26 mouse412803
27 mouse410315
28 mouse417678
29 mouse421529
30 mouse421338
31 mouse424448


In [20]:
#brain observatory
condi='cortex_nwb2'
#condi='all_units'
for mouse_idx, mouseID in enumerate(mouseIDs[5:]):
    print(mouse_idx, mouseID)
    #output = '/local1/work_allen/Ephys/resorted/'+mouseID
    output = '/local1/work_allen/Ephys/resorted/'+mouseID
    output_path = output+'/matrix/'

    output_meta = mouseID+'_'+condi+'_meta.csv'
    output_path = output+'/matrix/'
    
    key = 'flashes' 
    stim_table = pd.read_csv(output_path+'stim_table_'+key+'.csv')
    colors = nanunique(stim_table['color'].values)
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')
    
    # repeats for all conditions
    rep = int(np.shape(matrix_unit_binarized)[1]/(len(stim_table['color'].unique())))
    spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(stim_table['color'].unique()), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

    for idx_c, c in enumerate(stim_table['color'].unique()):
        tmp = matrix_unit_binarized[:,np.where((stim_table['color']==c))[0],:]
        spikes[:,idx_c,:,:] = tmp.astype('uint8')

    save_npz(spikes, output_path+'flash_'+condi+'_rep.npz')
    

0 mouse404551
1 mouse404571
2 mouse404555
3 mouse404569
4 mouse403407
5 mouse412791
6 mouse412792
7 mouse405755
8 mouse404570
9 mouse404554
10 mouse405751
11 mouse406807
12 mouse412794
13 mouse412793
14 mouse406805
15 mouse412799
16 mouse407972
17 mouse406808
18 mouse412802
19 mouse410343
20 mouse408152
21 mouse412803
22 mouse410315
23 mouse417678
24 mouse421529
25 mouse421338
26 mouse424448


In [19]:
#brain observatory
condi='cortex_nwb2'
#condi='all_units'
for mouse_idx, mouseID in enumerate(mouseIDs[5:]):
    print(mouse_idx, mouseID)
    #output = '/local1/work_allen/Ephys/resorted/'+mouseID
    output = '/local1/work_allen/Ephys/resorted/'+mouseID
    output_path = output+'/matrix/'

    output_meta = mouseID+'_'+condi+'_meta.csv'
    output_path = output+'/matrix/'
    
       
    # repeats for all conditions
    key = 'static_gratings' 
    # 1. static gratings
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    stim_table = pd.read_csv(output_path+'stim_table_'+key+'.csv')
    #nanunique sort conditions; otherwise if use unique() need to np.sort
    sfs = np.array([round(amo,2) for amo in nanunique(stim_table['spatial_frequency'])])
    oris = nanunique(stim_table['orientation'].values)
    phases = nanunique(stim_table['phase'].values)

    sf_all = np.array([round(amo,2) for amo in stim_table['spatial_frequency'].values])

    rep = int(np.shape(matrix_unit_binarized)[1]/(len(sfs)* len(oris)*len(phases)))
    rep=50
    spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(oris), len(sfs), len(phases), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

    for idx_o, o in enumerate(oris):
        for idx_s, s in enumerate(sfs):
            for idx_p, p in enumerate(phases):
                tmp = matrix_unit_binarized[:,np.where((stim_table['orientation']==o) & (sf_all==s) & (stim_table['phase']==p))[0],:]
                # pad nan if there is missing rep
                if np.shape(tmp)[1]<np.shape(spikes)[-2]:
                    result = np.zeros((np.shape(spikes)[0], np.shape(spikes)[-2], np.shape(spikes)[-1]))*np.NaN
                    result[:,:tmp.shape[1],:] = tmp.astype('uint8')
                    spikes[:,idx_o,idx_s,idx_p,:,:] = result.astype('uint8')
                else:
                    spikes[:,idx_o,idx_s,idx_p,:,:] = tmp.astype('uint8')

    idx_blank = np.where(np.isnan(stim_table['orientation'].values)==1)[0]
    spikes_blank = matrix_unit_binarized[:,idx_blank,:].astype('uint8')
    save_npz(np.squeeze(spikes), output_path+'static_grating_'+condi+'_rep.npz')


    #for movie with repeats
    key = 'natural_movie_one'
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    stim_table = pd.read_csv(output_path+'stim_table_'+key+'.csv')
    frame_times = stim_table.End-stim_table.Start
    n_frame = len(stim_table['frame'].unique())

    #bin_time = round(np.mean(frame_times)*1000)
    bin_time = 33
    delay=50 # need to change if onset time is corrected
    unit_psth, time = nwb_adapter_spikes.get_PSTH(matrix_unit_binarized[:,:,delay:int(bin_time*n_frame+delay)], bin_time)

    unit_psth = np.rollaxis(unit_psth, 2,1)
    print(np.shape(unit_psth))

    np.save(output_path+'movie_1_'+condi+'.npy',unit_psth)

    key = 'natural_movie_three'
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    stim_table = pd.read_csv(output_path+'stim_table_'+key+'.csv')
    frame_times = stim_table.End-stim_table.Start
    n_frame = len(stim_table['frame'].unique())

    #bin_time = round(np.mean(frame_times)*1000)
    bin_time = 33
    delay=50 # need to change if onset time is corrected
    unit_psth, time = nwb_adapter_spikes.get_PSTH(matrix_unit_binarized[:,:,delay:int(bin_time*n_frame+delay)], bin_time)

    unit_psth = np.rollaxis(unit_psth, 2,1)
    print(np.shape(unit_psth))

    np.save(output_path+'movie_3_'+condi+'.npy',unit_psth)

    
    #for images
    key = 'Natural Images_5'
    matrix_unit_binarized = load_npz(output_path+'natural_scenes'+'_'+condi+'.npz')

    stim_table = pd.read_csv(output_path+'stim_table_'+key+'.csv')
    # repeats for all conditions
    rep = int(np.shape(matrix_unit_binarized)[1]/(len(stim_table["b'frame'"].unique())))
    rep=50
    spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(stim_table["b'frame'"].unique()), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')
    images = np.sort(stim_table["b'frame'"].unique())
    for idx_i, i in enumerate(images):
        tmp = matrix_unit_binarized[:,np.where((stim_table["b'frame'"]==i))[0],:]
        spikes[:,idx_i,:,:] = tmp.astype('uint8')

    save_npz(np.squeeze(spikes), output_path+'image_'+condi+'.npz')

        

0 mouse404551
(434, 20, 29700)
(434, 900, 20)
(434, 10, 118800)
(434, 3600, 10)
1 mouse404571
(509, 20, 29700)
(509, 900, 20)
(509, 10, 118800)
(509, 3600, 10)
2 mouse404555
(497, 20, 29700)
(497, 900, 20)
(497, 10, 118800)
(497, 3600, 10)
3 mouse404569
(377, 20, 29700)
(377, 900, 20)
(377, 10, 118800)
(377, 3600, 10)
4 mouse403407
(555, 20, 29700)
(555, 900, 20)
(555, 10, 118800)
(555, 3600, 10)
5 mouse412791
(700, 20, 29700)
(700, 900, 20)
(700, 10, 118800)
(700, 3600, 10)
6 mouse412792
(741, 20, 29700)
(741, 900, 20)
(741, 10, 118800)
(741, 3600, 10)
7 mouse405755
(628, 20, 29700)
(628, 900, 20)
(628, 10, 118800)
(628, 3600, 10)
8 mouse404570
(561, 18, 29700)
(561, 900, 18)
(561, 10, 118800)
(561, 3600, 10)
9 mouse404554
(433, 20, 29700)
(433, 900, 20)
(433, 10, 118800)
(433, 3600, 10)
10 mouse405751
(794, 20, 29700)
(794, 900, 20)
(794, 10, 118800)
(794, 3600, 10)
11 mouse406807
(692, 20, 29700)
(692, 900, 20)
(692, 10, 118800)
(692, 3600, 10)
12 mouse412794
(629, 20, 29700)
(629, 

In [16]:
np.shape(spikes)[-2]

46

In [11]:
stim_table.keys()

Index(['Unnamed: 0', 'Start', 'End', 'b'frame''], dtype='object')

In [11]:
stim_table = pd.read_csv(output_path+'stim_table_'+key+'.csv')

In [20]:
tmp.shape

(369, 74, 250)

In [4]:
# functional connectivity
condi='cortex_nwb2'
#condi='all_units'
for mouse_idx, mouseID in enumerate(mouseIDs):
    print(mouse_idx, mouseID)
    #output = '/local1/work_allen/Ephys/resorted/'+mouseID
    output = '/local1/work_allen/Ephys/resorted/'+mouseID
    output_path = output+'/matrix/'

    output_meta = mouseID+'_'+condi+'_meta.csv'
    output_path = output+'/matrix/'
    
    key = 'flashes' 
    stim_table = pd.read_csv(output_path+'stim_table_'+key+'.csv')
    colors = nanunique(stim_table['color'].values)
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')
    
    # repeats for all conditions
    rep = int(np.shape(matrix_unit_binarized)[1]/(len(stim_table['color'].unique())))
    spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(stim_table['color'].unique()), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

    for idx_c, c in enumerate(stim_table['color'].unique()):
        tmp = matrix_unit_binarized[:,np.where((stim_table['color']==c))[0],:]
        spikes[:,idx_c,:,:] = tmp.astype('uint8')

    save_npz(spikes, output_path+'flash_'+condi+'_rep.npz')

    #for gratings
    key = 'drifting_gratings_75_repeats'
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    stim_table = pd.read_csv(output_path+'stim_table_'+key+'.csv')
    oris = np.sort(stim_table.orientation.unique())
    cons = np.sort(stim_table.contrast.unique())

    rep = int(np.shape(matrix_unit_binarized)[1]/(len(oris)*len(cons)))
    spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(oris), len(cons), int(rep), np.shape(matrix_unit_binarized)[2])).astype('uint8')

    for idx_o, o in enumerate(oris):
        for idx_c, c in enumerate(cons):
            tmp = matrix_unit_binarized[:,np.where((stim_table[stim_table.keys()[-1]]==o) & (stim_table['contrast']==c))[0],:]
            # pad 0 if there is missing rep
            if np.shape(tmp)[1]<np.shape(spikes)[-2]:
                result = np.zeros((np.shape(spikes)[0], np.shape(spikes)[-2], np.shape(spikes)[-1]))
                result[:,:tmp.shape[1],:] = tmp.astype('uint8')
                spikes[:,idx_o,idx_c,:,:] = result.astype('uint8')
            else:
                spikes[:,idx_o,idx_c,:,:] = tmp.astype('uint8')
    spikes = np.squeeze(spikes)
    save_npz(np.squeeze(spikes), output_path+'drifting_grating_'+condi+'_rep.npz')

    #for contrast
    key = 'drifting_gratings_contrast'
    stim_table = pd.read_csv(output_path+'stim_table_'+key+'.csv')
    # duration=0.5ms, blank=0.5ms

    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    oris = np.sort(stim_table.orientation.unique())
    cons = np.sort(stim_table.contrast.unique())

    rep = int(np.shape(matrix_unit_binarized)[1]/(len(oris)* len(cons)))
    spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(oris), len(cons), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

    for idx_o, o in enumerate(oris):
        for idx_c, c in enumerate(cons):
            tmp = matrix_unit_binarized[:,np.where((stim_table['orientation']==o) &  (stim_table['contrast']==c))[0],:]
            # pad 0 if there is missing rep
            if np.shape(tmp)[1]<np.shape(spikes)[-2]:
                result = np.zeros((np.shape(spikes)[0], np.shape(spikes)[-2], np.shape(spikes)[-1]))
                result[:,:tmp.shape[1],:] = tmp.astype('uint8')
                spikes[:,idx_o,idx_c,:,:] = result.astype('uint8')
            else:
                spikes[:,idx_o,idx_c,:,:] = tmp.astype('uint8')
                
    save_npz(np.squeeze(spikes), output_path+'contrast_'+condi+'_rep.npz')

    #coherent dots
    key = 'dot_motion'
    stim_table = pd.read_csv(output_path+'stim_table_'+key+'.csv')
    # duration=0.5ms, blank=0.5ms

    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    # 15 repeats
    oris = np.sort(stim_table.Dir.unique())
    speeds = np.sort(stim_table.Speed.unique())

    #rep = int(np.shape(matrix_unit_binarized)[1]/(len(oris)* len(speeds)))
    rep=15
    spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(oris), len(speeds), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

    for idx_o, o in enumerate(oris):
        for idx_s, s in enumerate(speeds):
            tmp = matrix_unit_binarized[:,np.where((stim_table['Dir']==o) &  (stim_table['Speed']==s))[0],:]
            # pad 0 if there is missing rep
            if np.shape(tmp)[1]<np.shape(spikes)[-2]:
                result = np.zeros((np.shape(spikes)[0], np.shape(spikes)[-2], np.shape(spikes)[-1]))
                result[:,:tmp.shape[1],:] = tmp.astype('uint8')
                spikes[:,idx_o,idx_s,:,:] = result.astype('uint8')
            else:
                spikes[:,idx_o,idx_s,:,:] = tmp.astype('uint8')
    #np.save(output_path+'coherent08_dot_'+condi+'.npy',spikes)
    save_npz(np.squeeze(spikes), output_path+'coherent08_dot_'+condi+'_rep.npz')

    #for movie with repeats
    key = 'natural_movie_one_more_repeats'
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    stim_table = pd.read_csv(output_path+'stim_table_'+key+'.csv')
    frame_times = stim_table.End-stim_table.Start
    n_frame = len(stim_table['frame'].unique())

    bin_time = round(np.mean(frame_times[:100])*1000)
    delay=70 # need to change if onset time is corrected
    unit_psth, time = nwb_adapter_spikes.get_PSTH(matrix_unit_binarized[:,:,delay:int(bin_time*n_frame+delay)], bin_time)

    unit_psth = np.rollaxis(unit_psth, 2,1)
    print(np.shape(unit_psth))

    np.save(output_path+'movie_'+condi+'_rep.npy',unit_psth)

    key = 'natural_movie_one_shuffled'
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    stim_table = pd.read_csv(output_path+'stim_table_'+key+'.csv')
    frame_times = stim_table.End-stim_table.Start
    n_frame = len(stim_table['frame'].unique())

    bin_time = round(np.mean(frame_times[:100])*1000)
    delay=70 # need to change if onset time is corrected
    unit_psth, time = nwb_adapter_spikes.get_PSTH(matrix_unit_binarized[:,:,delay:int(bin_time*n_frame+delay)], bin_time)

    unit_psth = np.rollaxis(unit_psth, 2,1)
    print(np.shape(unit_psth))

    np.save(output_path+'movie_shuffled_'+condi+'_rep.npy',unit_psth)


0 mouse419119
(721, 60, 29700)
(721, 900, 60)
(721, 20, 29700)
(721, 900, 20)
1 mouse424445
(557, 58, 29700)
(557, 900, 58)
(557, 20, 29700)
(557, 900, 20)
2 mouse415148
(842, 60, 29700)
(842, 900, 60)
(842, 20, 29700)
(842, 900, 20)
3 mouse416356
(598, 60, 29700)
(598, 900, 60)
(598, 20, 29700)
(598, 900, 20)
4 mouse416861
(865, 60, 29700)
(865, 900, 60)
(865, 20, 29700)
(865, 900, 20)
5 mouse419112
(797, 60, 29700)
(797, 900, 60)
(797, 20, 29700)
(797, 900, 20)
6 mouse419116
(776, 60, 29700)
(776, 900, 60)
(776, 20, 29700)
(776, 900, 20)
7 mouse434488
(691, 60, 29700)
(691, 900, 60)
(691, 20, 29700)
(691, 900, 20)
8 mouse437661
(689, 60, 29700)
(689, 900, 60)
(689, 20, 29700)
(689, 900, 20)
9 mouse448503
(912, 60, 29700)
(912, 900, 60)
(912, 20, 29700)
(912, 900, 20)
10 mouse433891
(718, 60, 29700)
(718, 900, 60)
(718, 20, 29700)
(718, 900, 20)
11 mouse432105
(580, 60, 29700)
(580, 900, 60)
(580, 20, 29700)
(580, 900, 20)
12 mouse432104
(608, 60, 29700)
(608, 900, 60)
(608, 20, 29700

In [41]:
tmp = load_npz(output_path+'drifting_grating_cortex_rep_nwb2.npz')
tmp.shape

(342, 4, 2, 75, 2000)

In [39]:
#BOC

(342, 1200000)

In [ ]:
condi='cortex'
#condi='all_units'
for mouse_idx, mouseID in enumerate(mouseIDs):
    print(mouse_idx, mouseID)
    output = '/local1/work_allen/Ephys/resorted/'+mouseID
    output_path = output+'/matrix/'

    output_meta = mouseID+'_cortex_meta.csv'
    output_path = output+'/matrix/'
    
    key='flashes'
    stim_table = pd.read_csv(output_path+'stim_table_'+key+'.csv')
    colors = nanunique(stim_table['color'].values)
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')
    
    # repeats for all conditions
    rep = int(np.shape(matrix_unit_binarized)[1]/(len(colors)))
    spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(colors), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

    for idx_c, c in enumerate(colors):
        tmp = matrix_unit_binarized[:,np.where((stim_table['color']==c))[0],:]
        spikes[:,idx_c,:,:] = tmp.astype('uint8')
        
    save_npz(spikes, output_path+'flash_'+condi+'_rep_nwb2.npz')
    
    # repeats for all conditions
    if 'static_grating*' in keys:
        key = 'static_gratings' 
        # 1. static gratings
        matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

        stim_table = test_obj.get_stim_table(key)
        #nanunique sort conditions; otherwise if use unique() need to np.sort
        sfs = np.array([round(amo,2) for amo in nanunique(stim_table['spatial_frequency'])])
        oris = nanunique(stim_table['orientation'].values)
        phases = nanunique(stim_table['phase'].values)

        sf_all = np.array([round(amo,2) for amo in stim_table['spatial_frequency'].values])

        rep = int(np.shape(matrix_unit_binarized)[1]/(len(sfs)* len(oris)*len(phases)))
        spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(oris), len(sfs), len(phases), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

        for idx_o, o in enumerate(oris):
            for idx_s, s in enumerate(sfs):
                for idx_p, p in enumerate(phases):
                    tmp = matrix_unit_binarized[:,np.where((stim_table['orientation']==o) & (sf_all==s) & (stim_table['phase']==p))[0],:]
                    # pad 0 if there is missing rep
                    if np.shape(tmp)[1]<np.shape(spikes)[-2]:
                        result = np.zeros((np.shape(spikes)[0], np.shape(spikes)[-2], np.shape(spikes)[-1]))
                        result[:,:tmp.shape[1],:] = tmp.astype('uint8')
                        spikes[:,idx_o,idx_s,idx_p,:,:] = result.astype('uint8')
                    else:
                        spikes[:,idx_o,idx_s,idx_p,:,:] = tmp.astype('uint8')

        idx_blank = np.where(np.isnan(stim_table['orientation'].values)==1)[0]
        spikes_blank = matrix_unit_binarized[:,idx_blank,:].astype('uint8')
        save_npz(np.squeeze(spikes), output_path+'static_grating_'+condi+'_rep_nwb2.npz')


    # drifting gratings
    key = 'drifting_gratings' 
    # 1. static gratings
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')
    stim_table = test_obj.get_stim_table(key)
    sfs = np.array([round(amo,2) for amo in nanunique(stim_table['spatial_frequency'].values)])
    oris = nanunique(stim_table['orientation'].values)
    tfs = np.array([round(amo,2) for amo in nanunique(stim_table['temporal_frequency'].values)])

    sf_all = np.array([round(amo,2) for amo in stim_table[b'spatial_frequency'].values])
    tf_all = np.array([round(amo,2) for amo in stim_table[b'temporal_frequency'].values])

    rep = int(np.shape(matrix_unit_binarized)[1]/(len(sfs)* len(oris)*len(tfs)))
    spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(oris), len(sfs), len(tfs), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

    for idx_o, o in enumerate(oris):
        for idx_s, s in enumerate(sfs):
            for idx_t, t in enumerate(tfs):
                tmp = matrix_unit_binarized[:,np.where((stim_table[b'orientation']==o) & (sf_all==s) & (tf_all==t))[0],:]
                # pad 0 if there is missing rep
                if np.shape(tmp)[1]<np.shape(spikes)[-2]:
                    result = np.zeros((np.shape(spikes)[0], np.shape(spikes)[-2], np.shape(spikes)[-1]))
                    result[:,:tmp.shape[1],:] = tmp.astype('uint8')
                    spikes[:,idx_o,idx_s,idx_t,:,:] = result.astype('uint8')
                else:
                    spikes[:,idx_o,idx_s,idx_t,:,:] = tmp.astype('uint8')

    idx_blank = np.where(np.isnan(stim_table[b'orientation'].values)==1)[0]
    spikes_blank = matrix_unit_binarized[:,idx_blank,:].astype('uint8')

    save_npz(np.squeeze(spikes), output_path+'drifting_grating_'+condi+'_rep_nwb2.npz')

    save_npz(np.squeeze(spikes_blank), output_path+'grating_blank_'+condi+'_rep_nwb2.npz')

    #for movie with repeats
    key = 'natural_movie_1_3'
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    stim_table = test_obj.get_stim_table(key) 
    frame_times = stim_table.End-stim_table.Start
    n_frame = len(stim_table[b'frame'].unique())

    #bin_time = round(np.mean(frame_times)*1000)
    bin_time = 33
    delay=50 # need to change if onset time is corrected
    unit_psth, time = nwb_adapter_spikes.get_PSTH(matrix_unit_binarized[:,:,delay:int(bin_time*n_frame+delay)], bin_time)

    unit_psth = np.rollaxis(unit_psth, 2,1)
    print(np.shape(unit_psth))

    np.save(output_path+'movie_1_'+condi+'_rep_nwb2.npy',unit_psth)

    key = 'natural_movie_3_4'
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    stim_table = test_obj.get_stim_table(key) 
    frame_times = stim_table.End-stim_table.Start
    n_frame = len(stim_table[b'frame'].unique())

    #bin_time = round(np.mean(frame_times)*1000)
    bin_time = 33
    delay=50 # need to change if onset time is corrected
    unit_psth, time = nwb_adapter_spikes.get_PSTH(matrix_unit_binarized[:,:,delay:int(bin_time*n_frame+delay)], bin_time)

    unit_psth = np.rollaxis(unit_psth, 2,1)
    print(np.shape(unit_psth))

    np.save(output_path+'movie_3_'+condi+'_rep_nwb2.npy',unit_psth)

    
    #for images
    key = 'Natural Images_5'
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    stim_table = test_obj.get_stim_table(key) 
    # repeats for all conditions
    rep = int(np.shape(matrix_unit_binarized)[1]/(len(stim_table[b'frame'].unique())))
    spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(stim_table[b'frame'].unique()), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')
    images = np.sort(stim_table[b'frame'].unique())
    for idx_i, i in enumerate(images):
        tmp = matrix_unit_binarized[:,np.where((stim_table[b'frame']==i))[0],:]
        spikes[:,idx_i,:,:] = tmp.astype('uint8')

    save_npz(np.squeeze(spikes), output_path+'image_'+condi+'_rep_nwb2.npz')

    if 'contrast_response_7' in keys:
        key = 'contrast_response_7'
        stim_table = test_obj.get_stim_table(key)
        # duration=0.5ms, blank=0.5ms

        matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

        oris = np.sort(stim_table[b'orientation'].unique())
        cons = np.sort(stim_table[b'Contrast'].unique())

        rep = int(np.shape(matrix_unit_binarized)[1]/(len(oris)* len(cons)))
        spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(oris), len(cons), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

        for idx_o, o in enumerate(oris):
            for idx_c, c in enumerate(cons):
                tmp = matrix_unit_binarized[:,np.where((stim_table[b'orientation']==o) &  (stim_table[b'Contrast']==c))[0],:]
                # pad 0 if there is missing rep
                if np.shape(tmp)[1]<np.shape(spikes)[-2]:
                    result = np.zeros((np.shape(spikes)[0], np.shape(spikes)[-2], np.shape(spikes)[-1]))
                    result[:,:tmp.shape[1],:] = tmp.astype('uint8')
                    spikes[:,idx_o,idx_c,:,:] = result.astype('uint8')
                else:
                    spikes[:,idx_o,idx_c,:,:] = tmp.astype('uint8')


        save_npz(np.squeeze(spikes), output_path+'contrast_'+condi+'_rep_nwb2.npz')

    if 'contrast_response_7' in keys:
        key = 'contrast_response_7'
        stim_table = test_obj.get_stim_table(key)
        # duration=0.5ms, blank=0.5ms

        matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

        oris = np.sort(stim_table[b'orientation'].unique())
        cons = np.sort(stim_table[b'Contrast'].unique())

        rep = int(np.shape(matrix_unit_binarized)[1]/(len(oris)* len(cons)))
        spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(oris), len(cons), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

        for idx_o, o in enumerate(oris):
            for idx_c, c in enumerate(cons):
                tmp = matrix_unit_binarized[:,np.where((stim_table[b'orientation']==o) &  (stim_table[b'Contrast']==c))[0],:]
                # pad 0 if there is missing rep
                if np.shape(tmp)[1]<np.shape(spikes)[-2]:
                    result = np.zeros((np.shape(spikes)[0], np.shape(spikes)[-2], np.shape(spikes)[-1]))
                    result[:,:tmp.shape[1],:] = tmp.astype('uint8')
                    spikes[:,idx_o,idx_c,:,:] = result.astype('uint8')
                else:
                    spikes[:,idx_o,idx_c,:,:] = tmp.astype('uint8')


        save_npz(np.squeeze(spikes), output_path+'contrast_'+condi+'_rep_nwb2.npz')

    if 'motion_stimulus_5' in keys:
        key = 'motion_stimulus_5'
        stim_table = test_obj.get_stim_table(key)
        # duration=0.5ms, blank=0.5ms

        matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

        # 15 repeats
        oris = np.sort(stim_table[b'direction'].unique())
        speeds = np.sort(stim_table[b'speed'].unique())

        #rep = int(np.shape(matrix_unit_binarized)[1]/(len(oris)* len(speeds)))
        rep=15
        spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(oris), len(speeds), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

        for idx_o, o in enumerate(oris):
            for idx_s, s in enumerate(speeds):
                tmp = matrix_unit_binarized[:,np.where((stim_table[b'direction']==o) &  (stim_table[b'speed']==s))[0],:]
                # pad 0 if there is missing rep
                if np.shape(tmp)[1]<np.shape(spikes)[-2]:
                    result = np.zeros((np.shape(spikes)[0], np.shape(spikes)[-2], np.shape(spikes)[-1]))
                    result[:,:tmp.shape[1],:] = tmp.astype('uint8')
                    spikes[:,idx_o,idx_s,:,:] = result.astype('uint8')
                else:
                    spikes[:,idx_o,idx_s,:,:] = tmp.astype('uint8')
        save_npz(np.squeeze(spikes), output_path+'coherent08_dot_'+condi+'_rep_nwb2.npz')
        
        

In [23]:
mouse_id = sessions_id.iloc[np.where(sessions_id.id.values==session_id)[0][0]].mouseID
mouseID='mouse'+str(mouse_id)

In [26]:
# all units in one session
# calculate response features
output = '/Volumes/local1/work_allen/Ephys/resorted/'+mouseID
output_path = output+'/matrix/'
output_meta = mouseID+'_all_units_meta.csv'
    
f = output_path+output_meta

df = session.units
df = df.rename(columns={"channel_local_index": "channel_id", 
                        "ecephys_structure_acronym": "ccf", 
                        "probe_id":"probe_global_id", 
                        "probe_description":"probe_id",
                       'probe_vertical_position': "ypos"})
df['unit_id']=df.index
# channel_id is not strictly sorted, but roughly sorted according to depth


/Users/xiaoxuanj/work/work_allen/Ephys/ecephys_SDK/AllenSDK/allensdk/brain_observatory/ecephys/ecephys_project_api/ecephys_project_warehouse_api.py:295: FutureWarning: Conversion of the second argument of issubdtype from `bool` to `np.generic` is deprecated. In future, it will be treated as `np.bool_ == np.dtype(bool).type`.
  pv_is_bool = np.issubdtype(output["p_value_rf"].values[0], bool)


In [ ]:
# order df by local channel index on each probe


# order matrix the same way



In [30]:
keys = session.stimulus_names # just the unique values from the 'stimulus_name' column
print(keys)

['spontaneous', 'gabors', 'flashes', 'drifting_gratings_contrast', 'natural_movie_one_more_repeats', 'natural_movie_one_shuffled', 'drifting_gratings_75_repeats', 'dot_motion']


In [ ]:
keys = session.stimulus_names # just the unique values from the 'stimulus_name' column
print(keys)

for key in keys:
    print(key)
    stim_table = session.get_stimulus_table([key])
    stim_table=stim_table.rename(columns={"start_time": "Start", "stop_time": "End"})
    if 'natural_movie' in key:
        frame_times = stim_table.End-stim_table.Start
        print('frame rate:', 1/np.mean(frame_times), 'Hz', np.mean(frame_times))
        # chunch each movie clip
        stim_table = stim_table[stim_table.frame==0]
        stim_table = stim_table.drop(['End'], axis=1)
        duration = np.mean(np.diff(stim_table.Start.values)[:10])
    elif key=='spontaneous':
        duration = stim_table.duration
        index = np.where(duration==max(duration))[0][0]
        if max(duration)>20: # only keep the longest spontaneous; has to be longer than 20 sec
            w=20
            start = np.array(list(stim_table.iloc[index].Start+np.arange(5,max(duration)-w,w)))
            end=start+w
            stim_table=pd.DataFrame()
            stim_table['Start']=start
            stim_table['End']=end
            duration=w
    else:
        #ISI = np.mean(session.get_inter_presentation_intervals_for_stimulus([key]).interval.values)
        #print(ISI)
        duration = round(np.mean(stim_table.duration.values), 2)
        print(duration)
    
    #binarize tensor
    # binarize with 1 second bins
    time_bin_edges = np.linspace(0, duration, int(duration*1000)+1)

    # and get a set of units with only decent snr
    #decent_snr_unit_ids = session.units[
    #    session.units['snr'] >= 1.5
    #].index.values
    cortex = ['VISp', 'VISl', 'VISli', 'VISrl', 'VISal', 'VISam', 'VISpm']
    cortical_units_ids = np.array([idx for idx, ccf in enumerate(df.ccf.values) if ccf in cortex])
    print(len(cortical_units_ids))

    # get binarized tensor
    df_cortex = df.iloc[cortical_units_ids]
    spike_counts_da = session.presentationwise_spike_counts(
        bin_edges=time_bin_edges,
        stimulus_presentation_ids=stim_table.index.values,
        unit_ids=df_cortex.unit_id.values
    )

    matrix = spike_counts_da.values
    matrix = np.rollaxis(matrix, -1,0)
    print(matrix.shape)
    assert len(df_cortex)==matrix.shape[0]
        
    plt.figure()
    plt.plot(matrix.mean(1).mean(1))
    
    matrix_2d = matrix.reshape(matrix.shape[0], int(len(matrix.flatten())/matrix.shape[0]))
    sparse_matrix = sp.csc_matrix(matrix_2d)    
    np.savez(output_path+key+'_cortex.npz',[sparse_matrix, matrix.shape])

    stim_table.to_csv(output_path+'stim_table_'+key+'.csv')
    
    if key=='drifting_gratings':
        df_cortex['FR']=(matrix[:,:,50:250].sum(2)/float(200)*1000).mean(1)  
        df_cortex.to_csv(f, encoding='utf-8')
    

['spontaneous', 'gabors', 'flashes', 'drifting_gratings_contrast', 'natural_movie_one_more_repeats', 'natural_movie_one_shuffled', 'drifting_gratings_75_repeats', 'dot_motion']
spontaneous
342
(342, 89, 20000)
gabors
0.25
342


/Users/xiaoxuanj/work/work_allen/Ephys/ecephys_SDK/AllenSDK/allensdk/brain_observatory/ecephys/ecephys_session.py:611: UserWarning: You've specified some overlapping time intervals between neighboring rows: [(0, 1)], with a maximum overlap of 0.016469278933342935 seconds.
  warnings.warn(f"You've specified some overlapping time intervals between neighboring rows: {overlapping}, "


(342, 3645, 250)
flashes
0.25
342
(342, 150, 250)
drifting_gratings_contrast
0.5
342
(342, 540, 500)
natural_movie_one_more_repeats
frame rate: 29.974929967739318 Hz 0.03336121222222222
342


/Users/xiaoxuanj/work/work_allen/Ephys/ecephys_SDK/AllenSDK/allensdk/brain_observatory/ecephys/ecephys_session.py:611: UserWarning: You've specified some overlapping time intervals between neighboring rows: [(0, 1), (1, 2), (3, 4), (5, 6), (7, 8), (11, 12), (13, 14), (14, 15), (16, 17), (17, 18), (19, 20), (22, 23), (27, 28), (30, 31), (31, 32), (33, 34), (34, 35), (36, 37), (37, 38), (41, 42), (42, 43), (43, 44), (44, 45), (46, 47), (47, 48), (49, 50), (50, 51), (52, 53), (56, 57), (57, 58), (58, 59)], with a maximum overlap of 5.400000009103678e-05 seconds.
  warnings.warn(f"You've specified some overlapping time intervals between neighboring rows: {overlapping}, "


(342, 60, 30025)
natural_movie_one_shuffled
frame rate: 29.97492497609337 Hz 0.03336121777777774
342


/Users/xiaoxuanj/work/work_allen/Ephys/ecephys_SDK/AllenSDK/allensdk/brain_observatory/ecephys/ecephys_session.py:611: UserWarning: You've specified some overlapping time intervals between neighboring rows: [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19)], with a maximum overlap of 360.3012229999995 seconds.
  warnings.warn(f"You've specified some overlapping time intervals between neighboring rows: {overlapping}, "


In [ ]:
# repeats for all conditions
rep = np.shape(matrix)[1]/(len(stim_table.color.unique()))
spikes = np.zeros((np.shape(matrix)[0], len(stim_table.color.unique()), rep, np.shape(matrix)[2])).astype('uint8')
for idx_c, c in enumerate(stim_table.color.unique()):
    tmp = matrix[:,np.where((stim_table['color']==c))[0],:]
    spikes[:,idx_c,:,:] = tmp.astype('uint8')


In [ ]:
condi='cortex'
#condi='all_units'
for mouse_idx, mouseID in enumerate(mouseIDs[29:]):
    print(mouse_idx, mouseID)
    output = '/local1/work_allen/Ephys/'+mouseID
    output_path = output+'/matrix/'
    
    nwbpath = '/local1/work_allen/Ephys/nwb_files/BOb_1_1'
    filename = mouseID+'.spikes.nwb'
    nwb_data = h5py.File(os.path.join(nwbpath,filename),'r')
    keys = list(nwb_data['stimulus']['presentation'].keys())
    #print(keys)

    # define output files
    test_obj = nwb_adapter_spikes.NWB_adapter(nwb_data)

    output_meta = mouseID+'_cortex_meta.csv'
    output_path = output+'/matrix/'
    
    key = 'flash_250ms_1' 
    stim_table = test_obj.get_stim_table(key)
    colors = nanunique(stim_table[b'color'].values)
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')
    
    # repeats for all conditions
    rep = int(np.shape(matrix_unit_binarized)[1]/(len(colors)))
    spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(colors), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

    for idx_c, c in enumerate(colors):
        tmp = matrix_unit_binarized[:,np.where((stim_table[b'color']==c))[0],:]
        spikes[:,idx_c,:,:] = tmp.astype('uint8')
        
    save_npz(spikes, output_path+'flash_'+condi+'.npz')
    
    # repeats for all conditions
    key = 'static_gratings_6' 
    # 1. static gratings
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    stim_table = test_obj.get_stim_table(key)
    #nanunique sort conditions; otherwise if use unique() need to np.sort
    sfs = np.array([round(amo,2) for amo in nanunique(stim_table[b'spatial_frequency'])])
    oris = nanunique(stim_table[b'orientation'].values)
    phases = nanunique(stim_table[b'phase'].values)

    sf_all = np.array([round(amo,2) for amo in stim_table[b'spatial_frequency'].values])

    rep = int(np.shape(matrix_unit_binarized)[1]/(len(sfs)* len(oris)*len(phases)))
    spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(oris), len(sfs), len(phases), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

    for idx_o, o in enumerate(oris):
        for idx_s, s in enumerate(sfs):
            for idx_p, p in enumerate(phases):
                tmp = matrix_unit_binarized[:,np.where((stim_table[b'orientation']==o) & (sf_all==s) & (stim_table[b'phase']==p))[0],:]
                # pad 0 if there is missing rep
                if np.shape(tmp)[1]<np.shape(spikes)[-2]:
                    result = np.zeros((np.shape(spikes)[0], np.shape(spikes)[-2], np.shape(spikes)[-1]))
                    result[:,:tmp.shape[1],:] = tmp.astype('uint8')
                    spikes[:,idx_o,idx_s,idx_p,:,:] = result.astype('uint8')
                else:
                    spikes[:,idx_o,idx_s,idx_p,:,:] = tmp.astype('uint8')

    idx_blank = np.where(np.isnan(stim_table[b'orientation'].values)==1)[0]
    spikes_blank = matrix_unit_binarized[:,idx_blank,:].astype('uint8')
    save_npz(np.squeeze(spikes), output_path+'static_grating_'+condi+'_rep.npz')


    # drifting gratings
    key = 'drifting_gratings_2' 
    # 1. static gratings
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')
    stim_table = test_obj.get_stim_table(key)
    sfs = np.array([round(amo,2) for amo in nanunique(stim_table[b'spatial_frequency'].values)])
    oris = nanunique(stim_table[b'orientation'].values)
    tfs = np.array([round(amo,2) for amo in nanunique(stim_table[b'temporal_frequency'].values)])

    sf_all = np.array([round(amo,2) for amo in stim_table[b'spatial_frequency'].values])
    tf_all = np.array([round(amo,2) for amo in stim_table[b'temporal_frequency'].values])

    rep = int(np.shape(matrix_unit_binarized)[1]/(len(sfs)* len(oris)*len(tfs)))
    spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(oris), len(sfs), len(tfs), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

    for idx_o, o in enumerate(oris):
        for idx_s, s in enumerate(sfs):
            for idx_t, t in enumerate(tfs):
                tmp = matrix_unit_binarized[:,np.where((stim_table[b'orientation']==o) & (sf_all==s) & (tf_all==t))[0],:]
                # pad 0 if there is missing rep
                if np.shape(tmp)[1]<np.shape(spikes)[-2]:
                    result = np.zeros((np.shape(spikes)[0], np.shape(spikes)[-2], np.shape(spikes)[-1]))
                    result[:,:tmp.shape[1],:] = tmp.astype('uint8')
                    spikes[:,idx_o,idx_s,idx_t,:,:] = result.astype('uint8')
                else:
                    spikes[:,idx_o,idx_s,idx_t,:,:] = tmp.astype('uint8')

    idx_blank = np.where(np.isnan(stim_table[b'orientation'].values)==1)[0]
    spikes_blank = matrix_unit_binarized[:,idx_blank,:].astype('uint8')

    save_npz(np.squeeze(spikes), output_path+'drifting_grating_'+condi+'_rep.npz')

    save_npz(np.squeeze(spikes_blank), output_path+'grating_blank_'+condi+'_rep.npz')

    #for movie with repeats
    key = 'natural_movie_1_3'
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    stim_table = test_obj.get_stim_table(key) 
    frame_times = stim_table.End-stim_table.Start
    n_frame = len(stim_table[b'frame'].unique())

    #bin_time = round(np.mean(frame_times)*1000)
    bin_time = 33
    delay=50 # need to change if onset time is corrected
    unit_psth, time = nwb_adapter_spikes.get_PSTH(matrix_unit_binarized[:,:,delay:int(bin_time*n_frame+delay)], bin_time)

    unit_psth = np.rollaxis(unit_psth, 2,1)
    print(np.shape(unit_psth))

    np.save(output_path+'movie_1_'+condi+'.npy',unit_psth)

    key = 'natural_movie_3_4'
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    stim_table = test_obj.get_stim_table(key) 
    frame_times = stim_table.End-stim_table.Start
    n_frame = len(stim_table[b'frame'].unique())

    #bin_time = round(np.mean(frame_times)*1000)
    bin_time = 33
    delay=50 # need to change if onset time is corrected
    unit_psth, time = nwb_adapter_spikes.get_PSTH(matrix_unit_binarized[:,:,delay:int(bin_time*n_frame+delay)], bin_time)

    unit_psth = np.rollaxis(unit_psth, 2,1)
    print(np.shape(unit_psth))

    np.save(output_path+'movie_3_'+condi+'.npy',unit_psth)

    
    #for images
    key = 'Natural Images_5'
    matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

    stim_table = test_obj.get_stim_table(key) 
    # repeats for all conditions
    rep = int(np.shape(matrix_unit_binarized)[1]/(len(stim_table[b'frame'].unique())))
    spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(stim_table[b'frame'].unique()), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')
    images = np.sort(stim_table[b'frame'].unique())
    for idx_i, i in enumerate(images):
        tmp = matrix_unit_binarized[:,np.where((stim_table[b'frame']==i))[0],:]
        spikes[:,idx_i,:,:] = tmp.astype('uint8')

    save_npz(np.squeeze(spikes), output_path+'image_'+condi+'.npz')

    if 'contrast_response_7' in keys:
        key = 'contrast_response_7'
        stim_table = test_obj.get_stim_table(key)
        # duration=0.5ms, blank=0.5ms

        matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

        oris = np.sort(stim_table[b'orientation'].unique())
        cons = np.sort(stim_table[b'Contrast'].unique())

        rep = int(np.shape(matrix_unit_binarized)[1]/(len(oris)* len(cons)))
        spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(oris), len(cons), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

        for idx_o, o in enumerate(oris):
            for idx_c, c in enumerate(cons):
                tmp = matrix_unit_binarized[:,np.where((stim_table[b'orientation']==o) &  (stim_table[b'Contrast']==c))[0],:]
                # pad 0 if there is missing rep
                if np.shape(tmp)[1]<np.shape(spikes)[-2]:
                    result = np.zeros((np.shape(spikes)[0], np.shape(spikes)[-2], np.shape(spikes)[-1]))
                    result[:,:tmp.shape[1],:] = tmp.astype('uint8')
                    spikes[:,idx_o,idx_c,:,:] = result.astype('uint8')
                else:
                    spikes[:,idx_o,idx_c,:,:] = tmp.astype('uint8')


        save_npz(np.squeeze(spikes), output_path+'contrast_'+condi+'.npz')

    if 'contrast_response_7' in keys:
        key = 'contrast_response_7'
        stim_table = test_obj.get_stim_table(key)
        # duration=0.5ms, blank=0.5ms

        matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

        oris = np.sort(stim_table[b'orientation'].unique())
        cons = np.sort(stim_table[b'Contrast'].unique())

        rep = int(np.shape(matrix_unit_binarized)[1]/(len(oris)* len(cons)))
        spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(oris), len(cons), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

        for idx_o, o in enumerate(oris):
            for idx_c, c in enumerate(cons):
                tmp = matrix_unit_binarized[:,np.where((stim_table[b'orientation']==o) &  (stim_table[b'Contrast']==c))[0],:]
                # pad 0 if there is missing rep
                if np.shape(tmp)[1]<np.shape(spikes)[-2]:
                    result = np.zeros((np.shape(spikes)[0], np.shape(spikes)[-2], np.shape(spikes)[-1]))
                    result[:,:tmp.shape[1],:] = tmp.astype('uint8')
                    spikes[:,idx_o,idx_c,:,:] = result.astype('uint8')
                else:
                    spikes[:,idx_o,idx_c,:,:] = tmp.astype('uint8')


        save_npz(np.squeeze(spikes), output_path+'contrast_'+condi+'.npz')

    if 'motion_stimulus_5' in keys:
        key = 'motion_stimulus_5'
        stim_table = test_obj.get_stim_table(key)
        # duration=0.5ms, blank=0.5ms

        matrix_unit_binarized = load_npz(output_path+key+'_'+condi+'.npz')

        # 15 repeats
        oris = np.sort(stim_table[b'direction'].unique())
        speeds = np.sort(stim_table[b'speed'].unique())

        #rep = int(np.shape(matrix_unit_binarized)[1]/(len(oris)* len(speeds)))
        rep=15
        spikes = np.zeros((np.shape(matrix_unit_binarized)[0], len(oris), len(speeds), rep, np.shape(matrix_unit_binarized)[2])).astype('uint8')

        for idx_o, o in enumerate(oris):
            for idx_s, s in enumerate(speeds):
                tmp = matrix_unit_binarized[:,np.where((stim_table[b'direction']==o) &  (stim_table[b'speed']==s))[0],:]
                # pad 0 if there is missing rep
                if np.shape(tmp)[1]<np.shape(spikes)[-2]:
                    result = np.zeros((np.shape(spikes)[0], np.shape(spikes)[-2], np.shape(spikes)[-1]))
                    result[:,:tmp.shape[1],:] = tmp.astype('uint8')
                    spikes[:,idx_o,idx_s,:,:] = result.astype('uint8')
                else:
                    spikes[:,idx_o,idx_s,:,:] = tmp.astype('uint8')
        save_npz(np.squeeze(spikes), output_path+'coherent08_dot_'+condi+'.npz')
        
        